In [159]:
#import all modules here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.linear_model as sk
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from random import randint
from IPython.display import display
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
import seaborn as sn
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import random

In [ ]:
# data = pd.read_csv("/home/shinigami/new/LabelNoise/haberman.data",header=None)
# data.fillna(data.median(),inplace=True)
# data.columns = ['Age','Year','Nodes','Survival_status']
# data['Survival_status'] = data['Survival_status'].astype(int)
# data['Survival_status'] = data['Survival_status'].map({2:0 , 1:1})
# data.drop_duplicates(inplace=True)
# label_col_name = 'Survival_status'

In [ ]:
# data = pd.read_csv('/home/shinigami/new/LabelNoise/SPECT.train',header=None)
# # features = df.loc[:,df.columns!=0]
# # label = df[0]
# label_col_name = 0

In [ ]:
# df = pd.read_csv('/home/shinigami/new/LabelNoise/heart-disease.data',header=None, names=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','label'])
# label = df['label']
# df.replace(r'?',np.nan,inplace=True)
# df.fillna(df.median(),inplace=True)
# df_std = StandardScaler().fit_transform(df)
# df = pd.DataFrame(df_std)
# df.head()
# df.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','label']
# df['label'] = label
# df['label'] = df['label'].map({1:1,2:1,3:1,4:1,0:0})
# data=df
# label_col_name = 'label'

In [ ]:
# data = pd.read_csv("/home/shinigami/new/LabelNoise/Binary Classified Datasets/ionosphere.csv")
# data.head(4)
# data.drop(['Unnamed: 0'],axis=1,inplace=True)
# label_col_name = 'g'

In [ ]:
data = pd.read_csv("/home/shinigami/new/LabelNoise/Binary Classified Datasets/data_banknote_authentication.txt",header=None)
data.head(3)
label_col_name=4
data[4].value_counts()

0    762
1    610
Name: 4, dtype: int64

In [ ]:
def cluster_as_a_feature(data,label_col_name,p,noc,model):
    ############################################
    data['Original_label'] = data[label_col_name]
    class0 = data.loc[data[label_col_name]!=1,:]
    class1 = data.loc[data[label_col_name]!=0,:]
    def add_noise(data,p):
        sample = data.sample(frac=p)
        for i in list(sample.index):
            data[label_col_name][i] = int(not data[label_col_name][i])
    add_noise(class0,p)
    add_noise(class1,p)
    data_df = pd.concat([class0,class1])
    data_df.reset_index(inplace=True,drop=True)
#     print(data_df.head(3))
    noisy_indices = data_df[data_df[label_col_name]!=data_df['Original_label']].index
#     print(list(noisy_indices))
#     ############################################
    km = KMeans(n_clusters=noc)
    features = data_df[data_df.columns.difference([label_col_name,'Original_label'])]
    label = data_df[label_col_name]
    km.fit(features)
    cluster_label = pd.Series(km.labels_)
    data_df['Cluster'] = cluster_label
    data_df = pd.get_dummies(data_df,columns=['Cluster'])
    features = data_df[data_df.columns.difference([label_col_name,'Original_label'])]
    label = data_df[label_col_name]
    xtrain,xtest,ytrain,ytest = train_test_split(features,label,test_size=0.38)
    m = model()
    m.fit(xtrain,ytrain)
    predict = m.predict(features)
    p2p = 0
    p2n = 0
    n2p = 0
    n2n = 0
    for i in list(data_df.index):
        if data_df[label_col_name][i]!=predict[i]:
            if i in noisy_indices:
                n2n += 1
            else:
                p2n += 1
        else:
            if i in noisy_indices:
                n2p += 1
            else:
                p2p += 1
    return pd.DataFrame(data={'n2n':[n2n],'p2n':[p2n],'n2p':[n2p],'p2p':[p2p]})

In [ ]:
l = []
models = [SVC,RandomForestClassifier,GaussianNB,LogisticRegression,DecisionTreeClassifier]
#result = pd.DataFrame(data={'Total Noise':[], 'Total Detected labels':[], 'True Identified labels':[], 'False identified labels':[], 'Total Noise Ratio':[], 'True Identified Ratio':[]})
result = pd.DataFrame()
for model in models:
    temp = cluster_as_a_feature(data,label_col_name,0.1,14,model)
    result = result.append(temp, ignore_index=True)
result['Model']=pd.Series(["SVC","Random Forest","Naive Bayes","Logistic Regression","Decision Tree"])
l.append(result)

In [ ]:
models = [SVC,RandomForestClassifier,GaussianNB,LogisticRegression,DecisionTreeClassifier]
#result = pd.DataFrame(data={'Total Noise':[], 'Total Detected labels':[], 'True Identified labels':[], 'False identified labels':[], 'Total Noise Ratio':[], 'True Identified Ratio':[]})
result = pd.DataFrame()
for model in models:
    temp = cluster_as_a_feature(data,label_col_name,0.2,14,model)
    result = result.append(temp, ignore_index=True)
result['Model']=pd.Series(["SVC","Random Forest","Naive Bayes","Logistic Regression","Decision Tree"])
l.append(result)

In [ ]:
models = [SVC,RandomForestClassifier,GaussianNB,LogisticRegression,DecisionTreeClassifier]
#result = pd.DataFrame(data={'Total Noise':[], 'Total Detected labels':[], 'True Identified labels':[], 'False identified labels':[], 'Total Noise Ratio':[], 'True Identified Ratio':[]})
result = pd.DataFrame()
for model in models:
    temp = cluster_as_a_feature(data,label_col_name,0.3,14,model)
    result = result.append(temp, ignore_index=True)
result['Model']=pd.Series(["SVC","Random Forest","Naive Bayes","Logistic Regression","Decision Tree"])
l.append(result)

In [ ]:
models = [SVC,RandomForestClassifier,GaussianNB,LogisticRegression,DecisionTreeClassifier]
#result = pd.DataFrame(data={'Total Noise':[], 'Total Detected labels':[], 'True Identified labels':[], 'False identified labels':[], 'Total Noise Ratio':[], 'True Identified Ratio':[]})
result = pd.DataFrame()
for model in models:
    temp = cluster_as_a_feature(data,label_col_name,0.4,14,model)
    result = result.append(temp, ignore_index=True)
result['Model']=pd.Series(["SVC","Random Forest","Naive Bayes","Logistic Regression","Decision Tree"])
l.append(result)

In [ ]:
models = [SVC,RandomForestClassifier,GaussianNB,LogisticRegression,DecisionTreeClassifier]
#result = pd.DataFrame(data={'Total Noise':[], 'Total Detected labels':[], 'True Identified labels':[], 'False identified labels':[], 'Total Noise Ratio':[], 'True Identified Ratio':[]})
result = pd.DataFrame()
for model in models:
    temp = cluster_as_a_feature(data,label_col_name,0.5,14,model)
    result = result.append(temp, ignore_index=True)
result['Model']=pd.Series(["SVC","Random Forest","Naive Bayes","Logistic Regression","Decision Tree"])
l.append(result)

In [ ]:
(pd.concat(l)).to_csv("cluster_as_a_feature_banknote.csv")